In [8]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from sklearn.metrics import mean_squared_error

In [6]:
# hyperparameters
window_size = 500
stride = 5
# threshold = 0.1

In [7]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(stride, 500)))
model.add(RepeatVector(stride))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(500)))
model.compile(optimizer='adam', loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 64)                144640    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 5, 64)             0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 64)             33024     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 500)            32500     
Total params: 210,164
Trainable params: 210,164
Non-trainable params: 0
_________________________________________________________________
